In [3]:
import glob,pylab,pandas as pd
import pydicom,numpy as np
import random
import json
import time
import copy
import torchvision
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from matplotlib import patches,patheffects

from sklearn.model_selection import train_test_split
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.optim import lr_scheduler
from pathlib import Path

PATH = Path("/data/krf/dataset")

class CDataset(Dataset):
    def __init__(self,ds,img_dir,class_df = None,transform=None,ext=None):
        self.ds = ds
        self.img_dir = img_dir
        self.class_df = class_df
        self.ext = ext or '.dcm'
        self.transform = transforms.Compose(transform) if transform else None
    
    def __len__(self):
        return len(self.ds)
    
    def read_dicom_image(self,loc):
        img_arr = pydicom.read_file(loc.as_posix()).pixel_array
        img_arr = img_arr/img_arr.max()
        img_arr = (255*img_arr).clip(0,255).astype(np.uint8)
        img_arr = Image.fromarray(img_arr).convert('RGB')
        return img_arr
    
    def __getitem__(self,i):
        img = self.read_dicom_image(self.ds[i])
        if self.transform:
            img = self.transform(img)
        patientId = self.ds[i].name.split('.')[0]
        #kls = self.class_df[self.class_df['patientId'] == patientId]
        return img,patientId#,kls.iloc[0].Target



use_gpu = torch.cuda.is_available()
print("Use gpu = {}".format(use_gpu))

#dataloaders = {'train':train_dl,'val':test_dl}

device = torch.cuda.set_device(0)


model_ft = torchvision.models.resnet18(pretrained=False)#True)
#修改分类数
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,2)

criterion = nn.CrossEntropyLoss()
model_ft = model_ft.cuda()

optimizer_ft = optim.Adam(model_ft.parameters())

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,step_size=7,gamma=0.1)




Use gpu = True


In [4]:
model_ft.load_state_dict(torch.load('resnet.ckpt'))

In [5]:
img_dir = PATH/'stage_1_test_images'
#img_dir = PATH/'stage_1_test_images'
#sample = random.sample(list(img_dir.iterdir()),100)
sample = list(img_dir.iterdir())

transform = [transforms.Resize(224),transforms.RandomHorizontalFlip(),transforms.ToTensor()]
test_ds = CDataset(sample,img_dir,transform=transform)

batch_size=1
sz=224
test_dl = DataLoader(test_ds,batch_size=batch_size)


In [1]:
count = 0
num = 1000
model_ft.eval()
labelFile = 'stage_1_test_labels.csv'
result = []
from tqdm import tqdm
with open(labelFile, 'w') as file:
    file.write("patientId,Target\n")
    
    for data in tqdm(test_dl):
        inputs,patientId = data
        #model_ft.train(False)
        if use_gpu:
            inputs = Variable(inputs.cuda(),requires_grad=True)
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        #optimizer.zero_grad()
        #print("inputs:",inputs)
        #print("patientId:",patientId)
        #forward
        outputs = model_ft(inputs)
        #print("outputs",outputs)
        _,preds = torch.max(outputs.data,1)
        if preds == 0:
            #print(0)
            result.append(0)
            file.write(patientId[0]+",0\n")
        else:
            #print(1)
            result.append(1)
            file.write(patientId[0]+",1\n")
        count+=1
        if count>=num:
            break
print(sum(result))

NameError: name 'model_ft' is not defined

In [6]:
##TEST ON TRAINING SET
train_bb_df = pd.read_csv(PATH/'stage_1_train_labels.csv')
#print(train_bb_df.head())

train_bb_df['duplicate'] = train_bb_df.duplicated(['patientId'],keep=False)
#print(train_bb_df[train_bb_df['duplicate']].head())

detailed_df = pd.read_csv(PATH/'stage_1_detailed_class_info.csv')
# merge two df
class_df = train_bb_df.merge(detailed_df,on="patientId")

csv_df = class_df.filter(['patientId','Target'],)
csv_df = csv_df.set_index('patientId',)
#class_df.head(10)
#print(csv_df.head(10))
class TDataset(Dataset):
    def __init__(self,ds,img_dir,class_df,transform=None,ext=None):
        self.ds = ds
        self.img_dir = img_dir
        self.class_df = class_df
        self.ext = ext or '.dcm'
        self.transform = transforms.Compose(transform) if transform else None
    
    def __len__(self):
        return len(self.ds)
    
    def read_dicom_image(self,loc):
        img_arr = pydicom.read_file(loc.as_posix()).pixel_array
        img_arr = img_arr/img_arr.max()
        img_arr = (255*img_arr).clip(0,255).astype(np.uint8)
        img_arr = Image.fromarray(img_arr).convert('RGB')
        return img_arr
    
    def __getitem__(self,i):
        img = self.read_dicom_image(self.ds[i])
        if self.transform:
            img = self.transform(img)
        patientId = self.ds[i].name.split('.')[0]
        kls = self.class_df[self.class_df['patientId'] == patientId]
        return img,kls.iloc[0].Target

img_dir = PATH/'stage_1_train_images'
sample = random.sample(list(img_dir.iterdir()),1000)
#sample = list(img_dir.iterdir())

transform = [transforms.Resize(224),transforms.RandomHorizontalFlip(),transforms.ToTensor()]
train_ds = TDataset(sample,img_dir,class_df,transform=transform)

batch_size=100
sz=224
train_dl = DataLoader(train_ds,batch_size = batch_size)


count = 0
num = 100
model_ft.eval()
#labelFile = 'stage_1_train_labels_pred.csv'
result = []
from tqdm import tqdm
running_corrects = 0
for data in tqdm(train_dl):
    inputs,labels = data
    #print(data)
    #print(labels)
    #model_ft.train(False)
    if use_gpu:
        inputs = Variable(inputs.cuda(),requires_grad=True)
        labels = Variable(labels.cuda())
    else:
        inputs,labels = Variable(inputs),Variable(labels)
        
        #print("inputs:",inputs)
        #print("patientId:",patientId)
        #forward
    print(inputs.shape)
    outputs = model_ft(inputs)
    #print("outputs",outputs)
    _,preds = torch.max(outputs.data,1)
    running_corrects += torch.sum(preds == labels.data)
    
print(running_corrects)
print(sum(result))

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([100, 3, 224, 224])
outputs 

 10%|█         | 1/10 [00:19<02:59, 19.92s/it]

tensor([[ 0.7550, -0.8063],
        [-0.9455,  0.6670],
        [ 2.9659, -3.9412],
        [ 0.5233, -0.5785],
        [ 2.2332, -2.8215],
        [ 1.0050, -1.1902],
        [ 2.8417, -3.8217],
        [ 1.0574, -1.1719],
        [ 1.7487, -2.0800],
        [ 0.6498, -0.6819],
        [ 3.0455, -4.1296],
        [ 2.9819, -4.0743],
        [ 2.4180, -3.1848],
        [ 1.0601, -1.1594],
        [-0.1360, -0.0129],
        [ 0.9436, -1.0720],
        [ 2.8283, -3.8423],
        [ 0.5188, -0.5591],
        [ 0.8064, -0.7059],
        [ 1.8847, -2.4305],
        [ 3.3112, -4.5301],
        [ 0.8992, -0.9676],
        [ 1.6524, -2.0065],
        [ 0.7263, -0.7705],
        [-0.6534,  0.4520],
        [ 2.3952, -3.1494],
        [ 0.8283, -0.8128],
        [ 2.7352, -3.6496],
        [ 1.6761, -2.0558],
        [ 0.7443, -0.7058],
        [ 1.4513, -1.7138],
        [ 2.7957, -3.8385],
        [-0.1199,  0.0494],
        [-1.0332,  0.7356],
        [ 3.0191, -4.1386],
        [-0.7424,  0

KeyboardInterrupt: 